In [5]:
import daw_readout
import process_data
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import sys
form time import time
import os

In [2]:
#file_list = 'runlist/darkrate_monitor_with_GECO2020'
file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'

In [3]:
path_save =""
winfo =[]
i=0
with open(file_list, 'r') as list:
    for line in list:    
        rawfilename = line.rstrip('\n')[17 :] 
        rawdata = daw_readout.DAWDemoWaveParser(rawfilename)     
        for wave in tqdm(rawdata) :
            ch = wave.Channel
            ttt = wave.Timestamp
            base = wave.Baseline
            pulse = wave.Waveform
            st, minp, ed = process_data.pusle_index(pulse)
            ht = base - wave.Waveform[minp]
            rst = minp  - st
            flt = ed - minp
            maxp = base - np.max(pulse)
            asy = ht / (ht + maxp)
            area = process_data.pusle_area(pulse, st, ed, base)
            area_fixlen = process_data.pulse_area_fix_len(pulse, st, 10, base)
            winfo.append({
                'EvtID': i,
                'Ch':ch,
                'TTT':ttt,   ## Trigger time tag
                'Baseline': base, 
                'Hight': ht, 
                'Width': ed -st,  ## pusle width
                'Minp': minp,     ## min point
                'Rst' : rst,      ## rise time of pulse    
                'Flt' : flt,      ## full time of pulse  
                'Asym' : asy,     ## asymmetry of pulse
                'Area': area,     ## area of pulse cal by dynamic range
                'AreaFixlen': area_fixlen,    ## area of pulse cal by fix length of 10
                'Wave': pulse,
            })
            i +=1
            
        file_tag = line.rstrip('\n')[17 :].rstrip('.bin')[24:][: -13]  
        path_save = "outnpy/{}.h5".format(file_tag)

0it [00:00, ?it/s]/tmp/ipykernel_3335424/3053002512.py:18: RuntimeWarning: divide by zero encountered in scalar divide
  asy = ht / (ht + maxp)
213270it [00:06, 31135.38it/s]
651372it [00:20, 31928.63it/s]


In [6]:
df = pd.DataFrame(winfo)
process_data.write_to_hdf5(df, path_save)

NameError: name 'time' is not defined